In [2]:
import numpy as np

class Board:
    def __init__(self):
        self.board=np.full(shape=(8,8), fill_value="e")
        self.board[4][4] = "w"
        self.board[4][3] = "b"
        self.board[3][3] = "w"
        self.board[3][4] = "b"

GAME_STATE = True
EMPTY = "e"
WHITE = "w"
BLACK = "b"
DIRECTIONS = ((0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1))

def opposite_color(color):
    if color == BLACK:
        return WHITE
    else:
        return BLACK

def is_valid_move(board, player, row, col):
    if board[row][col] != 'e':
        return False

    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            r, c = row + i, col + j
            if r < 0 or r >= 8 or c < 0 or c >= 8:
                continue
            if board[r][c] == opposite_color(player):
                while True:
                    r, c = r + i, c + j
                    if r < 0 or r >= 8 or c < 0 or c >= 8:
                        break
                    if board[r][c] == player:
                        return True
                    if board[r][c] == 'e':
                        break

    return False

def get_valid_moves(board, player):
    valid_moves = []
    for row in range(8):
        for col in range(8):
            if is_valid_move(board, player, row, col):
                valid_moves.append((row, col))
    return valid_moves

def flip_pieces(board, player, row, col):
    for direction in DIRECTIONS:
        r, c = row + direction[0], col + direction[1]
        if r < 0 or r >= 8 or c < 0 or c >= 8 or board[r][c] == player or board[r][c] == EMPTY:
            continue
        pieces_to_flip = []
        while board[r][c] != player:
            pieces_to_flip.append((r, c))
            r += direction[0]
            c += direction[1]
            if r < 0 or r >= 8 or c < 0 or c >= 8 or board[r][c] == EMPTY:
                break
        else:
            for piece in pieces_to_flip:
                board[piece[0]][piece[1]] = player
    board[row][col] = player

def get_winner(board):
    white_count = np.count_nonzero(board == WHITE)
    black_count = np.count_nonzero(board == BLACK)
    if white_count > black_count:
        return WHITE
    elif black_count > white_count:
        return BLACK
    else:
        return EMPTY

def make_move(board, player, row, col):
    if row < 0 or row >= len(board) or col < 0 or col >= len(board[0]):
        return False
    if not is_valid_move(board, player, row, col):
        return False
    flip_pieces(board, player, row, col)
    return True

def get_score(board):
    black_count = 0
    white_count = 0
    
    for row in range(8):
        for col in range(8):
            if board[row][col] == 'b':
                black_count += 1
            elif board[row][col] == 'w':
                white_count += 1
    
    return (black_count, white_count)

In [61]:
import pygame
import sys
# from board import Board
# from constants import BLACK, WHITE
# from game import get_valid_moves, make_move
# from utils import opposite_color


WINDOW_SIZE = [450, 500]
# Define some colors
BLACK_COLOR = (0, 0, 0)
WHITE_COLOR = (255, 255, 255)
GREEN_COLOR = (0, 255, 0)
RED_COLOR = (255, 0, 0)

SQUARE_SIZE = 56.25

# This sets the WIDTH and HEIGHT of each grid location
WIDTH = 50
HEIGHT = 50

# This sets the margin between each cell
MARGIN = 5

current_player = BLACK
clock = pygame.time.Clock()
pygame.init()
screen = pygame.display.set_mode(WINDOW_SIZE)
font = pygame.font.SysFont(None, 24)

# Set the title of the window
pygame.display.set_caption("Reversi Game")

# Loop until the user clicks the close button
done = False

def draw_lines():
    # Draw the horizontal grid lines
    for i in range(8):
        start_pos = (0, i * SQUARE_SIZE + SQUARE_SIZE )
        end_pos = (450, i * SQUARE_SIZE + SQUARE_SIZE)
        pygame.draw.line(screen, BLACK_COLOR, start_pos, end_pos)

    # Draw the vertical grid lines
    for i in range(8):
        start_pos = (i * SQUARE_SIZE + SQUARE_SIZE, 0)
        end_pos = (i * SQUARE_SIZE + SQUARE_SIZE, 450)
        pygame.draw.line(screen, BLACK_COLOR, start_pos, end_pos)

def is_game_over(board, current_player):
    black_moves = get_valid_moves(board, BLACK)
    white_moves = get_valid_moves(board, WHITE)
    if current_player == "b" and len(black_moves) == 0:
        return True
    elif current_player == "w" and len(white_moves) == 0:
        return True
    else:
        return False

def show_winner(screen, winner):
    font = pygame.font.Font(None, 50)
    text = font.render(f"Winner: {winner}", True, RED_COLOR)
    text_rect = text.get_rect(center=(WINDOW_SIZE[0] // 2, 50))
    screen.blit(text, text_rect)

def get_score_minmax(board, player):
    # Define weights for the board positions
    weights = np.array([
        [100, -30, 10, 8, 8, 10, -30, 100],
        [-30, -50, -4.5, -5, -5, -4.5, -50, -30],
        [10, -4.5, 0.3, 0.1, 0.1, 0.3, -4.5, 10],
        [8, -5, 0.1, 0.5, 0.5, 0.1, -5, 8],
        [8, -5, 0.1, 0.5, 0.5, 0.1, -5, 8],
        [10, -4.5, 0.3, 0.1, 0.1, 0.3, -4.5, 10],
        [-30, -50, -4.5, -5, -5, -4.5, -50, -30],
        [100, -30, 10, 8, 8, 10, -30, 100]
    ])

    # Get the scores for each player
    player_score = np.sum(board == player)
    opponent_score = np.sum(board == opposite_color(player))

    # Compute the final score
    corner_score = np.sum(weights * (board==player))
    return player_score - opponent_score + corner_score

def show_menu(screen):
    """
    Displays a menu and waits for the user to select an option.
    Returns '1' for a new game or '2' to quit.
    
    """
    screen.fill(BLACK_COLOR)
    font = pygame.font.Font(None, 36)
    title_text = font.render("Menu", True, WHITE_COLOR)
    new_game_text = font.render("Against player", True, WHITE_COLOR)
    ai1_game_text = font.render("Against AI start 1", True, WHITE_COLOR)
    ai2_game_text = font.render("Against AI start 2", True, WHITE_COLOR)
    quit_text = font.render("Quit", True, WHITE_COLOR)

    title_rect = title_text.get_rect(center=(WINDOW_SIZE[0] // 2, 100))
    new_game_rect = new_game_text.get_rect(center=(WINDOW_SIZE[0] // 2, 200))

    ai1_game_rect = ai1_game_text.get_rect(center=(WINDOW_SIZE[0] // 2, 250))
    ai2_game_rect = ai2_game_text.get_rect(center=(WINDOW_SIZE[0] // 2, 300))

    quit_rect = quit_text.get_rect(center=(WINDOW_SIZE[0] // 2, 350))

    screen.blit(title_text, title_rect)
    screen.blit(new_game_text, new_game_rect)
    screen.blit(quit_text, quit_rect)

    screen.blit(ai1_game_text, ai1_game_rect)
    screen.blit(ai2_game_text, ai2_game_rect)
    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                if new_game_rect.collidepoint(mouse_pos):
                    return '1'
                elif quit_rect.collidepoint(mouse_pos):
                    return '2'
                elif ai1_game_rect.collidepoint(mouse_pos):
                    return '3'
                elif ai2_game_rect.collidepoint(mouse_pos):
                    return '4'
def render_score(surface, font, score):
    if current_player == "b":
        player = "Black"
    else:
        player = "White"
    black_score, white_score = score
    text = f"Black score: {black_score}   White score: {white_score} Now: {player}"
    text_surface = font.render(text, True, (0, 0, 0))
    surface.blit(text_surface, (10, 450))

def draw_board(screen, board):
    for row in range(8):
        for col in range(8):
            x = col * SQUARE_SIZE
            y = row * SQUARE_SIZE
            if board[row][col] == BLACK:
                pygame.draw.circle(screen, BLACK_COLOR, (x+SQUARE_SIZE//2, y+SQUARE_SIZE//2), SQUARE_SIZE//2 - 5)
            elif board[row][col] == WHITE:
                pygame.draw.circle(screen, WHITE_COLOR, (x+SQUARE_SIZE//2, y+SQUARE_SIZE//2), SQUARE_SIZE//2 - 5)

def minimax(board, depth, player, max_player, min_player):
    if depth == 0:
        # if max_player =
        return None, get_score_minmax(board, player)
    
    if player == max_player:
        best_score = float("-inf")
        best_move = None
        for move in get_valid_moves(board, player):
            new_board = np.copy(board)
            make_move(new_board, player, move[0], move[1])
            score = minimax(new_board, depth-1, opposite_color(player), max_player, min_player)[1]
            if score > best_score:
                best_score = score
                best_move = move
        return best_move, best_score
    
    else:
        best_score = float("inf")
        best_move = None
        for move in get_valid_moves(board, player):
            new_board = np.copy(board)
            make_move(new_board, player, move[0], move[1])
            score = minimax(new_board, depth-1, opposite_color(player), max_player, min_player)[1]
            if score < best_score:
                best_score = score
                best_move = move
        return best_move, best_score


while True:
    option = show_menu(screen)
    if option == '1':
        board = Board()
        current_player = 'b'
        winner = None
        done=False

        # --- Game logic should go here
        while not done:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    # Get the position of the mouse
                    pos = pygame.mouse.get_pos()
                    # Convert the position into board coordinates
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    # Make a move if it is valid
                    if make_move(board.board, current_player, row, column):
                        # Switch the player
                        current_player = opposite_color(current_player)

            # --- Game logic should go here
            if is_game_over(board.board, current_player):
                # Get the scores
                black_score, white_score = get_score(board.board)

                # Determine the winner
                if black_score > white_score:
                    winner = "Black"
                elif white_score > black_score:
                    winner = "White"
                else:
                    winner = "Tie"

            screen.fill(GREEN_COLOR)
            font = pygame.font.Font(None, 24)
            button_text = font.render("Return to Menu", True, BLACK_COLOR)
            button_rect = button_text.get_rect(center=(WINDOW_SIZE[0] // 2, 475))
            screen.blit(button_text, button_rect)
            mouse_pos = pygame.mouse.get_pos()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if button_rect.collidepoint(mouse_pos):
                    done = True

            # --- Drawing code should go here
            # First, clear the screen to white. Don't put other drawing commands
            # above this, or they will be erased with this command.

            # Draw the grid lines
            draw_lines()

            # Draw the pieces on the board
            draw_board(screen, board.board)

            # Render the current score
            render_score(screen, font, get_score(board.board))

            # If the game is over, show the winner
            if winner:
                show_winner(screen, winner)

            # --- Go ahead and update the screen
            pygame.display.flip()

            # --- Limit to 60 frames per second
            clock.tick(60)

    # --- Quit the game ---
    elif option == '2':
        pygame.quit()
    elif option == '3':
        board = Board()
        current_player = 'b'
        winner = None
        done=False

        # --- Game logic should go here
        while not done:
            # button_rect.collidepoint(pygame.mouse.get_pos())


            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                elif current_player == BLACK:
                    move, score = minimax(board.board, 4, current_player, BLACK, WHITE)
                    make_move(board.board, current_player, move[0], move[1])
                    current_player = opposite_color(current_player)
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    # Get the position of the mouse
                    pos = pygame.mouse.get_pos()
                    # Convert the position into board coordinates
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    # Make a move if it is valid
                    if current_player==WHITE and make_move(board.board, current_player, row, column):
                        # Switch the player
                        current_player = opposite_color(current_player)

            # --- Game logic should go here
            if is_game_over(board.board, current_player):
                # Get the scores
                black_score, white_score = get_score(board.board)

                # Determine the winner
                if black_score > white_score:
                    winner = "Black"
                elif white_score > black_score:
                    winner = "White"
                else:
                    winner = "Tie"

            # --- Drawing code should go here
            # First, clear the screen to white. Don't put other drawing commands
            # above this, or they will be erased with this command.
            screen.fill(GREEN_COLOR)
            font = pygame.font.Font(None, 24)
            button_text = font.render("Return to Menu", True, BLACK_COLOR)
            button_rect = button_text.get_rect(center=(WINDOW_SIZE[0] // 2, 475))
            screen.blit(button_text, button_rect)
            mouse_pos = pygame.mouse.get_pos()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if button_rect.collidepoint(mouse_pos):
                    done = True

            # Draw the grid lines
            draw_lines()

            # Draw the pieces on the board
            draw_board(screen, board.board)

            # Render the current score
            render_score(screen, font, get_score(board.board))

            # If the game is over, show the winner
            if winner:
                show_winner(screen, winner)

            # --- Go ahead and update the screen
            pygame.display.flip()

            # --- Limit to 60 frames per second
            clock.tick(60)
    elif option == '4':
        board = Board()
        current_player = 'b'
        winner = None
        done = False

        # --- Game logic should go here
        while not done:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                elif current_player == WHITE:
                    move, score = minimax(board.board, 4, current_player, WHITE, BLACK)
                    print(move, score)
                    make_move(board.board, current_player, move[0], move[1])
                    current_player = opposite_color(current_player)
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    # Get the position of the mouse
                    pos = pygame.mouse.get_pos()
                    # Convert the position into board coordinates
                    column = pos[0] // (WIDTH + MARGIN)
                    row = pos[1] // (HEIGHT + MARGIN)
                    # Make a move if it is valid
                    if current_player==BLACK and make_move(board.board, current_player, row, column):
                        # Switch the player
                        current_player = opposite_color(current_player)

            # --- Game logic should go here
            if is_game_over(board.board, current_player):
                # Get the scores
                black_score, white_score = get_score(board.board)

                # Determine the winner
                if black_score > white_score:
                    winner = "Black"
                elif white_score > black_score:
                    winner = "White"
                else:
                    winner = "Tie"

            screen.fill(GREEN_COLOR)
            font = pygame.font.Font(None, 24)
            button_text = font.render("Return to Menu", True, BLACK_COLOR)
            button_rect = button_text.get_rect(center=(WINDOW_SIZE[0] // 2, 475))
            screen.blit(button_text, button_rect)
            mouse_pos = pygame.mouse.get_pos()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if button_rect.collidepoint(mouse_pos):
                    done = True

            # --- Drawing code should go here
            # First, clear the screen to white. Don't put other drawing commands
            # above this, or they will be erased with this command.

            # Draw the grid lines
            draw_lines()

            # Draw the pieces on the board
            draw_board(screen, board.board)

            # Render the current score
            render_score(screen, font, get_score(board.board))

            # If the game is over, show the winner
            if winner:
                show_winner(screen, winner)

            # --- Go ahead and update the screen
            pygame.display.flip()

            # --- Limit to 60 frames per second
            clock.tick(60)

    else:
        pygame.quit()
pygame.quit()


error: display Surface quit